<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0e4e21d3eee6c355df329ca1ec4286a8bc0e071264a11b8339d1ada9d33dc637
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 10:24:54
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -2.65 L (-1.84%)
Current PnL: -26.94 L (-17.4%)
CY Booked + Current PnL: -12.65 L (-8.17%)
-------------------
Total profit:  1.33 L
Total loss:  -28.28 L
-------------------
Total Booked + Current PnL: 14.33 L (11.21%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.11%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.57 L (64.79%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.16,-8.53,9.35,0.02,14928.0,-14897.0,159662.0,143.30,67.0,M-SC,15.52,234.0,-1.00,1.17,38.88,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.27,-16.94,20.50,0.09,17160.0,-17068.0,83709.0,94.24,36.0,M-SC,2.74,253.0,-0.99,0.61,9.98,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.20,-1.81,24.29,22.05,23369.0,-1770.0,96210.0,-15.54,60.0,H-SC,7.03,164.0,-0.08,0.71,39.85,XR,ATH,FINANCE
43,ITC,452.00,0.15,-1.38,11.98,10.44,23647.0,-2753.0,197385.0,-41.15,47.0,X-LC,1.57,5.0,-0.12,1.45,4.11,X40,NTT,FMCG
35,ICICIGI,2252.93,0.51,5.76,13.65,20.19,25166.0,10043.0,184363.0,-17.20,47.0,X-MC,6.62,68.0,0.40,1.35,21.75,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,6.68,-43.39,207.64,74.15,192308.0,-70989.0,92616.0,-266.56,46.0,H-SC,19.93,149.0,-0.37,0.68,18.72,XR,NTT,CHEMICALS
73,TCS,4389.97,2.17,-11.72,35.12,19.28,107255.0,-40559.0,305397.0,-22.98,71.0,X-LC,6.52,1.0,-0.38,2.24,12.90,X40,ATH,IT
29,HAPPSTMNDS,1480.71,2.15,-36.60,191.56,84.85,155654.0,-46909.0,81256.0,-24.68,53.0,H-SC,14.03,147.0,-0.30,0.60,4.29,AR,ATH,IT
24,EASEMYTRIP,26.40,1.83,-53.92,216.17,45.70,176133.0,-95336.0,81479.0,-15.39,63.0,M-SC,23.27,205.0,-0.54,0.60,17.44,XY24,NTT,TRAVEL
32,HCLTECH,1922.01,1.80,8.38,15.09,24.73,39564.0,20266.0,262190.0,13.18,69.0,X-LC,5.23,7.0,0.51,1.92,24.89,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-80.12,-79.89,520.54,24.79,263643.0,-201208.0,50648.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
44,JCHAC,2282.0,-4.74,-34.63,52.97,-0.01,44252.0,-44264.0,83541.0,16986.96,8.0,M-SC,1.01,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
51,MEDANTA,1486.0,-4.15,-2.97,27.87,24.08,34331.0,-3767.0,123183.0,-10.78,33.0,X-SC,6.02,89.0,-0.11,0.90,16.07,XY24,NTT,HEALTHCARE
28,GREENPANEL,537.0,-3.82,-37.23,128.04,43.14,147142.0,-68159.0,114919.0,157.11,26.0,M-SC,7.20,240.0,-0.46,0.84,8.10,XY24,NTT,MISC
33,HINDUNILVR,2922.0,-3.80,-7.86,23.36,13.67,55330.0,-20203.0,236860.0,-14.78,36.0,X-LC,3.33,14.0,-0.37,1.74,11.82,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,-1.99,-0.96,50.07,48.63,99152.0,-1927.0,198027.0,-34.11,58.0,H-SC,14.76,136.0,-0.02,1.45,10.48,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.63,-6.64,122.69,107.91,169124.0,-9797.0,137847.0,-24.20,31.0,M-SC,11.23,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.16,-8.53,9.35,0.02,14928.0,-14897.0,159662.0,143.30,67.0,M-SC,15.52,234.0,-1.00,1.17,38.88,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.27,-16.94,20.50,0.09,17160.0,-17068.0,83709.0,94.24,36.0,M-SC,2.74,253.0,-0.99,0.61,9.98,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.37,-24.19,49.68,13.47,101563.0,-65232.0,204435.0,-68.90,30.0,H-SC,2.40,158.0,-0.64,1.50,3.95,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,1.18,-24.55,74.98,32.03,74586.0,-32361.0,99474.0,-18.39,47.0,H-SC,7.86,148.0,-0.43,0.73,11.42,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.67,6.09,62.38,72.27,112136.0,10314.0,179762.0,-8.8,71.0,M-LC,2.96,99.0,0.09,1.32,12.64,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.67,6.09,62.38,72.27,112136.0,10314.0,179762.0,-8.80,71.0,M-LC,2.96,99.0,0.09,1.32,12.64,XR,NTT,IT
38,INDIAMART,4810.62,-1.34,-1.24,109.32,106.72,133157.0,-1531.0,121805.0,-51.01,36.0,H-SC,1.56,139.0,-0.01,0.89,21.85,AR,ATH,MISC
37,IEX,219.00,-1.99,-0.96,50.07,48.63,99152.0,-1927.0,198027.0,-34.11,58.0,H-SC,14.76,136.0,-0.02,1.45,10.48,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.32,-1.72,38.01,35.63,78286.0,-3613.0,205961.0,-15.41,40.0,H-MC,3.42,119.0,-0.05,1.51,14.66,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.63,-6.64,122.69,107.91,169124.0,-9797.0,137847.0,-24.20,31.0,M-SC,11.23,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.12,-79.89,520.54,24.79,263643.0,-201208.0,50648.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.0,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.67,-21.22,88.43,48.45,139688.0,-42541.0,157965.0,-26.96,12.0,X-MC,9.42,64.0,-0.30,1.16,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.92,-38.16,118.63,35.20,94396.0,-49098.0,79572.0,4.26,22.0,X-SC,14.44,92.0,-0.52,0.58,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.13,-12.79,39.42,21.58,48176.0,-17928.0,122211.0,-56.03,24.0,X-MC,5.78,56.0,-0.37,0.90,8.2,X40N,ATH,IT
54,PGHH,17907.65,-0.35,-5.92,42.18,33.76,79689.0,-11895.0,188925.0,-33.58,28.0,X-MC,4.69,57.0,-0.15,1.39,0.0,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.37,-23.72,115.21,64.15,209106.0,-56451.0,181500.0,-55.44,41.0,X-SC,1.49,82.0,-0.27,1.33,1.81,XY24,BTT,CEMENT
8,AWL,485.0,0.95,-20.91,94.08,53.50,224526.0,-63097.0,238654.0,-61.72,32.0,X-MC,1.56,58.0,-0.28,1.75,6.00,XY24,NTT,FMCG
43,ITC,452.0,0.15,-1.38,11.98,10.44,23647.0,-2753.0,197385.0,-41.15,47.0,X-LC,1.57,5.0,-0.12,1.45,4.11,X40,NTT,FMCG
15,CAMS,4762.0,-80.12,-79.89,520.54,24.79,263643.0,-201208.0,50648.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.15,2.50,27.77,30.97,50309.0,4424.0,181162.0,-21.60,53.0,X-MC,2.24,75.0,0.09,1.33,15.38,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.92,-38.16,118.63,35.20,94396.0,-49098.0,79572.0,4.26,22.0,X-SC,14.44,92.0,-0.52,0.58,0.0,X40,NTT,FOOTWEAR
15,CAMS,4762.00,-80.12,-79.89,520.54,24.79,263643.0,-201208.0,50648.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.0,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.67,-21.22,88.43,48.45,139688.0,-42541.0,157965.0,-26.96,12.0,X-MC,9.42,64.0,-0.30,1.16,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-2.09,-31.58,108.77,42.84,48378.0,-20529.0,44477.0,-55.37,34.0,X-SC,37.35,83.0,-0.42,0.33,0.0,XY24,NTT,MISC
54,PGHH,17907.65,-0.35,-5.92,42.18,33.76,79689.0,-11895.0,188925.0,-33.58,28.0,X-MC,4.69,57.0,-0.15,1.39,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.09,-31.58,108.77,42.84,48378.0,-20529.0,44477.0,-55.37,34.0,X-SC,37.35,83.0,-0.42,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-80.12,-79.89,520.54,24.79,263643.0,-201208.0,50648.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
59,RELAXO,1176.00,-0.31,-47.56,190.44,52.31,144965.0,-69039.0,76121.0,-44.15,34.0,X-SC,5.38,91.0,-0.48,0.56,1.89,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.92,-38.16,118.63,35.20,94396.0,-49098.0,79572.0,4.26,22.0,X-SC,14.44,92.0,-0.52,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.13,-12.79,39.42,21.58,48176.0,-17928.0,122211.0,-56.03,24.0,X-MC,5.78,56.0,-0.37,0.90,8.20,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,2.17,-11.72,35.12,19.28,107255.0,-40559.0,305397.0,-22.98,71.0,X-LC,6.52,1.0,-0.38,2.24,12.90,X40,ATH,IT
41,INFY,2275.00,0.86,9.70,41.18,54.87,141341.0,30340.0,343228.0,-14.57,70.0,X-LC,3.25,2.0,0.21,2.52,18.78,X40,BTT,IT
75,TMPV,600.00,-0.52,-15.97,69.20,42.18,160726.0,-44147.0,232263.0,-24.30,31.0,X-LC,3.96,3.0,-0.27,1.70,0.61,XY24,NTT,AUTO
81,VBL,671.64,-0.20,-3.74,40.94,35.67,124474.0,-11803.0,304039.0,-15.14,57.0,X-LC,5.32,4.0,-0.09,2.23,9.60,X40N,ATH,FMCG
43,ITC,452.00,0.15,-1.38,11.98,10.44,23647.0,-2753.0,197385.0,-41.15,47.0,X-LC,1.57,5.0,-0.12,1.45,4.11,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.64,-36.45,109.12,32.89,53538.0,-28144.0,49063.0,-706.04,64.0,L-MC,6.20,259.0,-0.53,0.36,35.11,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.39,-17.11,119.16,81.67,92661.0,-16048.0,77762.0,6988.89,38.0,L-SC,16.28,271.0,-0.17,0.57,50.59,XR,NTT,CERAMICS
50,MASFIN,398.61,0.20,-1.81,24.29,22.05,23369.0,-1770.0,96210.0,-15.54,60.0,H-SC,7.03,164.0,-0.08,0.71,39.85,XR,ATH,FINANCE
13,BSOFT,831.70,0.22,-20.80,90.39,50.79,100692.0,-29256.0,111397.0,1.66,75.0,H-SC,5.20,151.0,-0.29,0.82,29.70,XR,ATH,IT
69,SURYODAY,216.00,-2.21,-21.51,56.75,23.03,79766.0,-38515.0,140556.0,53.51,44.0,H-SC,10.05,167.0,-0.48,1.03,39.46,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.16,-8.53,9.35,0.02,14928.0,-14897.0,159662.0,143.30,67.0,M-SC,15.52,234.0,-1.00,1.17,38.88,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.01,3.36,17.02,20.95,44289.0,8448.0,260216.0,2.32,79.0,X-LC,18.46,31.0,0.19,1.91,40.19,X40,ATH,PAINTS
13,BSOFT,831.70,0.22,-20.80,90.39,50.79,100692.0,-29256.0,111397.0,1.66,75.0,H-SC,5.20,151.0,-0.29,0.82,29.70,XR,ATH,IT
41,INFY,2275.00,0.86,9.70,41.18,54.87,141341.0,30340.0,343228.0,-14.57,70.0,X-LC,3.25,2.0,0.21,2.52,18.78,X40,BTT,IT
32,HCLTECH,1922.01,1.80,8.38,15.09,24.73,39564.0,20266.0,262190.0,13.18,69.0,X-LC,5.23,7.0,0.51,1.92,24.89,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.64
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.17
MC    30.05
LC    24.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      22.39
X40N     10.81
XR       10.09
AR        9.41
XY25      9.33
OX40N     7.78
SR        1.03
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.64
X-MC    23.34
X-LC    21.19
M-SC    12.20
X-SC     6.93
H-MC     4.94
M-MC     1.41
L-SC     1.40
M-LC     1.32
H-LC     1.23
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.34,-6.49,40.93
IT,12.80,-15.54,78.01
FINANCE,9.62,-18.18,68.11
ELECTRICAL,6.26,-9.96,50.23
PAINTS,6.23,-8.73,24.88
MISC,6.13,-55.09,118.63
INSURANCE,4.54,-2.20,38.22
PHARMA,4.08,-0.53,32.70
AUTO,3.44,-22.31,74.75


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3170514.0,21
AR,1289202.0,10
XR,1284101.0,13
X40,985756.0,14
X40N,974100.0,9
OX40N,728749.0,10
XY25,369796.0,6
SR,280252.0,2
MH,74925.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3617727.0,25
M-SC,1391492.0,15
X-MC,1301180.0,16
X-SC,973745.0,8
X-LC,874063.0,11
H-MC,405791.0,3
L-SC,269347.0,3
M-LC,112136.0,1
H-LC,66126.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270016.0      6
           AR         887044.0      5
M-SC       XY24       794414.0      6
H-SC       XR         767599.0      7
X-SC       X40N       502832.0      3
X-MC       X40        450562.0      7
           XY24       386465.0      3
X-LC       X40        356096.0      5
H-SC       OX40N      337891.0      4
M-SC       OX40N      303069.0      5
X-SC       XY24       291815.0      3
H-SC       SR         280252.0      2
X-MC       X40N       273658.0      4
X-LC       XY24       233306.0      2
H-MC       AR         211293.0      2
X-LC       X40N       197610.0      2
H-MC       XY24       194498.0      1
X-MC       XY25       190495.0      2
L-SC       XR         181558.0      2
X-SC       X40        179098.0      2
M-SC       XR         169270.0      2
           AR         124739.0      2
M-LC       XR         112136.0      1
L-SC       OX40N       87789.0      1
X-LC       XY25        87051.0      2
H-SC       MH          74925.0      1
H-LC       AR          66126.0      1
L-MC       XR          53538.0      1
M-MC       XY25        52803.0      1
L-LC       XY25        39447.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
